# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading weather_data csv file
# Dropping the "Unnamed: 0" column
# Printing the DataFrame
weather_df = pd.read_csv("weather_data.csv").dropna()
weather_df.drop(weather_df.columns[weather_df.columns.str.contains('Unnamed: 0',case = False)],axis = 1, inplace = True)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed
0,East London,0,ZA,1630560020,93,-33.0153,27.9116,52.74,2.30
1,Hengshui,3,CN,1630560021,60,37.7322,115.7011,80.51,5.08
2,Ossora,98,RU,1630560021,79,59.2353,163.0719,57.54,9.82
3,Provideniya,90,RU,1630560022,76,64.3833,-173.3000,48.31,6.71
4,Ushuaia,0,AR,1630559951,75,-54.8000,-68.3000,37.06,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configuring maps
gmaps.configure(api_key=g_key)

# humidity heatmap
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df['Humidity'].astype(int)


# Plotting heatmap
fig = gmaps.figure()

# Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Adding layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Creating new DataFrame by copying previous DataFrame
weather_city_df = weather_df.copy()
weather_city_df = weather_city_df[weather_city_df['Maximum_Temperature'] >= 70]
weather_city_df = weather_city_df[weather_city_df['Maximum_Temperature'] <= 80]  
weather_city_df = weather_city_df[weather_city_df['Wind_Speed'] <= 10]
weather_city_df = weather_city_df[weather_city_df['Cloudiness'] == 0]

In [6]:
# Dropping rows that contain missing values 
# Printing the updated DataFrame
weather_city_df = weather_city_df.dropna()
weather_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed
8,Aydūn,0,JO,1630560024,69,32.5072,35.8575,72.81,5.86
26,Hobart,0,AU,1630560032,29,-42.8794,147.3294,78.06,5.01
33,New Norfolk,0,AU,1630560035,19,-42.7826,147.0587,77.72,7.00
74,Terrasini,0,IT,1630560054,75,38.1460,13.0844,76.21,4.61
77,Jalu,0,LY,1630560055,54,29.0331,21.5482,75.87,6.62


In [7]:
# Resetting the index
# Printing the updated DataFrame
weather_city_df  = weather_city_df.reset_index(drop=True)
weather_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed
0,Aydūn,0,JO,1630560024,69,32.5072,35.8575,72.81,5.86
1,Hobart,0,AU,1630560032,29,-42.8794,147.3294,78.06,5.01
2,New Norfolk,0,AU,1630560035,19,-42.7826,147.0587,77.72,7.00
3,Terrasini,0,IT,1630560054,75,38.1460,13.0844,76.21,4.61
4,Jalu,0,LY,1630560055,54,29.0331,21.5482,75.87,6.62


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Creating DataFrame called "hotel_df"
# Adding missing values/non values to the DataFrame using a column called "Hotel_Name"
# Dropping the index
# Printing the DataFrame
hotel_df = weather_city_df
hotel_df["Hotel_Name"] = np.nan
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed,Hotel_Name
0,Aydūn,0,JO,1630560024,69,32.5072,35.8575,72.81,5.86,NaN
1,Hobart,0,AU,1630560032,29,-42.8794,147.3294,78.06,5.01,NaN
2,New Norfolk,0,AU,1630560035,19,-42.7826,147.0587,77.72,7.00,NaN
3,Terrasini,0,IT,1630560054,75,38.1460,13.0844,76.21,4.61,NaN
4,Jalu,0,LY,1630560055,54,29.0331,21.5482,75.87,6.62,NaN


In [9]:
# Setting parameters to search for hotels with 5000 meters.
# Creating an empty list hotels
hotels = []

x = 0

# Creating a for loop to search trhough cities' coordinates
for cities in hotel_df['City']:
    coordinates = str(hotel_df['Latitude'][x])+' , '+str(hotel_df['Longitude'][x])
    x+=1
    radius = 5000
    type = 'lodging'
    params = {
    "location": coordinates,
    "radius": radius,
    "type": type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# Using except to check if there is a city available
    try:
        response = requests.get(base_url, params=params)
        results = response.json()
        hotels.append(results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

# Appending my DataFrame with hotels' names
hotel_df['Hotel_Name'] = hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed,Hotel_Name
0,Aydūn,0,JO,1630560024,69,32.5072,35.8575,72.81,5.86,Nahas Plaza
1,Hobart,0,AU,1630560032,29,-42.8794,147.3294,78.06,5.01,Mantra on Collins Hobart
2,New Norfolk,0,AU,1630560035,19,-42.7826,147.0587,77.72,7.00,The Shingles Riverside Cottages
3,Terrasini,0,IT,1630560054,75,38.1460,13.0844,76.21,4.61,Florio Park Hotel
4,Jalu,0,LY,1630560055,54,29.0331,21.5482,75.87,6.62,صاحبة الفخامة
5,Avera,0,US,1630560088,82,33.1940,-82.5271,76.71,6.11,NaN
6,Çermik,0,TR,1630560092,32,38.1361,39.4493,73.89,2.75,Çermik Öğretmen Evi
7,Līsakovsk,0,KZ,1630560119,33,52.5369,62.4936,73.44,9.31,Gostinitsa Lisakovsk
8,Sivota,0,GR,1630560127,62,39.4078,20.2452,73.22,7.29,Hotel Filakas
9,Victoria Point,0,AU,1630560147,62,-27.5833,153.3000,75.43,5.99,Bay Retreat Motel


In [10]:
# Removing NaN values
# Printing DataFrame
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum_Temperature,Wind_Speed,Hotel_Name
0,Aydūn,0,JO,1630560024,69,32.5072,35.8575,72.81,5.86,Nahas Plaza
1,Hobart,0,AU,1630560032,29,-42.8794,147.3294,78.06,5.01,Mantra on Collins Hobart
2,New Norfolk,0,AU,1630560035,19,-42.7826,147.0587,77.72,7.00,The Shingles Riverside Cottages
3,Terrasini,0,IT,1630560054,75,38.1460,13.0844,76.21,4.61,Florio Park Hotel
4,Jalu,0,LY,1630560055,54,29.0331,21.5482,75.87,6.62,صاحبة الفخامة


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Adding marker layer ontop of heat map
# Displaying figure
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))